#**Step 1: Install All the Required Packages**

In [ ]:
# GPU llama-cpp-python
!CMAKE_ARGS="-DLLAMA_CUBLAS=on" FORCE_CMAKE=1 pip install llama-cpp-python==0.1.78 numpy==1.23.4 --force-reinstall --upgrade --no-cache-dir --verbose
!pip install huggingface_hub
!pip install llama-cpp-python==0.1.78
!pip install numpy==1.23.4

Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 28.3 MB/s eta 0:00:00
  Running command pip subprocess to install build dependencies
  Using pip 23.1.2 from /usr/local/lib/python3.10/dist-packages/pip (python 3.10)
    Using cached setuptools-69.2.0-py3-none-any.whl (821 kB)
    Using cached scikit_build-0.17.6-py3-none-any.whl (84 kB)
    Using cached cmake-3.28.3-py2.py3-none-manylinux2014_x86_64.manylinux_2_17_x86_64.whl (26.3 MB)
    Using cached ninja-1.11.1.1-py2.py3-none-manylinux1_x86_64.manylinux_2_5_x86_64.whl (307 kB)
    Using cached distro-1.9.0-py3-none-any.whl (20 kB)
    Using cached packaging-24.0-py3-none-any.whl (53 kB)
    Using cached tomli-2.0.1-py3-none-any.whl (12 kB)
    Using cached wheel-0.43.0-py3-none-any.whl (65 kB)
    Creating /tmp/pip-build-env-wdjhs409/overlay/local/bin
    changing mode of /tmp/pip-build-env-wdjhs409/overlay/local/bin/ninja to 755
    changing mod

In [ ]:
model_name_or_path = "TheBloke/Llama-2-13B-chat-GGML"
model_basename = "llama-2-13b-chat.ggmlv3.q5_1.bin" # the model is in bin format

#**Step 2: Import All the Required Libraries**

In [ ]:
from huggingface_hub import hf_hub_download


In [ ]:
from llama_cpp import Llama


#**Step 3: Download the Model**

In [ ]:
model_path = hf_hub_download(repo_id=model_name_or_path, filename=model_basename)

#**Step 4: Loading the Model**

In [ ]:
# GPU
lcpp_llm = None
lcpp_llm = Llama(
    model_path=model_path,
    n_threads=2, # CPU cores
    n_batch=512, # Should be between 1 and n_ctx, consider the amount of VRAM in your GPU.
    n_gpu_layers=32 # Change this value based on your model and your GPU VRAM pool.
    )

AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | VSX = 0 | 


In [ ]:
# See the number of layers in GPU
lcpp_llm.params.n_gpu_layers

32

#**Step 5: Create a Prompt Template**

In [ ]:
prompt = "10 + 23 ?"
prompt_template=f'''SYSTEM: You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?', you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line.

USER: {prompt}

ASSISTANT:
'''

#**Step 6: Generating the Response**

In [ ]:
response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                  repeat_penalty=1.2, top_k=150,
                  echo=True)

In [ ]:
print(response)

{'id': 'cmpl-d0cee7f9-a751-484e-840f-471b22f4edcf', 'object': 'text_completion', 'created': 1710367749, 'model': '/root/.cache/huggingface/hub/models--TheBloke--Llama-2-13B-chat-GGML/snapshots/3140827b4dfcb6b562cd87ee3d7f07109b014dd0/llama-2-13b-chat.ggmlv3.q5_1.bin', 'choices': [{'text': "SYSTEM: You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?', you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line.\n\nUSER: 10 + 23 ?\n\nASSISTANT:\n\n10 + 23 = 33", 'index': 0, 'logprobs': None, 'finish_reason': 'stop'}], 'usage': {'prompt_tokens': 90, 'completion_tokens': 11, 'total_tokens': 101}}


In [ ]:
print(response["choices"][0]["text"])


SYSTEM: You are a math assistant. I will ask you some addition questions. Please answer in the correct format. For example, if I ask 'What is 2 + 3?', you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line.

USER: 10 + 23 ?

ASSISTANT:

10 + 23 = 33


In [ ]:
text=response["choices"][0]["text"]

In [ ]:
text.split('=')[-1].strip()

'33'

In [ ]:
import json
import os
import time


In [ ]:
#load file in data folder int_addition.json
with open('int_addition.json') as f:
    data = json.load(f)

In [ ]:
print(len(data))

24000


In [ ]:
data[343]

[-8, -5, -13]

In [ ]:
data=data[344:]

In [ ]:
print(len(data))

23656


In [ ]:
#data[0]=[2,1,3]
#for each element in data create a prompt and get the response and write it to a text file

import time
#time sleep to avoid rate limit

for i in range(len(data)):
    if i!=0 and i%100==0:
      print(f'processing {i}th element')
      #download the int_addition.txt file from the colab
      from google.colab import files
      files.download('int_addition_results.txt')
      time.sleep(20)
    try:


        prompt = f"{data[i][0]} + {data[i][1]} ?"
        prompt_template=f'''SYSTEM: You are a math assistant. I will ask you some addition questions. For example, if I ask 'What is 2 + 3?', you should answer '2 + 3 = 5'. Each question is in a separate line. Please return each answer in a separate line.
        USER: {prompt}

        ASSISTANT:
        '''

        response=lcpp_llm(prompt=prompt_template, max_tokens=256, temperature=0.5, top_p=0.95,
                            repeat_penalty=1.2, top_k=150,
                            echo=True)
        #for each response extract the answer and write it to a file
        with open('int_addition_results.txt', 'a') as f:
            answer = response["choices"][0]["text"]
            #extract the answer from the response. #there are 2 equal signs in the response use equal sign in assistant response to extract the answer
            answer = answer.split('=')[-1]
            answer = answer.strip()
            #save answer in format [2, 1, 3] = 3
            f.write(f'{data[i]} = {answer}\n')
    except:
        print('error in processing the prompt')
        time.sleep(2)
    #time sleep to avoid rate limit
    #what is the rate limit for llama-cpp-python

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

processing 100th element


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

...

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.gene

processing 600th element


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit
Llama.generate: prefix-match hit


In [ ]:
with open('int_addition.txt', 'r') as f:
    print(f.read())

[2, 1, 3] = 3
[4, 4, 8] = 8
[2, 9, 11] = 11
[7, 1, 8] = 8
[2, 4, 6] = 6
[9, 1, 10] = 10
[4, 9, 13] = 13
[4, 8, 12] = 12
[5, 1, 6] = 6
[7, 6, 13] = 13

